# NETTOYAGE DES DONNÉES

## Préalables

In [1]:
# Importation des librairies :
import numpy as np
import pandas as pd

In [2]:
# Importations des fichiers :
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
transactions = pd.read_csv("transactions.csv")

## I - Nettoyage du fichier "customers" :

In [3]:
customers.isnull().value_counts()
# Aucune valeur manquante.

client_id  sex    birth
False      False  False    8623
dtype: int64

### A - Colonne "client_id" :

In [4]:
customers["client_id"].duplicated().sort_values()
# Pas de doublons dans la colonne client_id.duplicated.

0       False
5756    False
5755    False
5754    False
5753    False
        ...  
2867    False
2866    False
2865    False
2879    False
8622    False
Name: client_id, Length: 8623, dtype: bool

In [5]:
customers["client_id"].sort_values()
# On remarque que deux entrées débutent par "ct" et non par "c". C'est peut-être une erreur d'irregularité.
# Après étude du fichier "transactions", il s'avère que "ct_0" et "ct_1" sont des clients test.

# Suppression des clients test :
clientsTest = ["ct_0", "ct_1"]
customers.drop(customers.loc[customers["client_id"].isin(clientsTest)].index, inplace=True)

### B - Colonne "sex" :

In [6]:
customers["sex"].unique()
# Il n'existe que deux valeurs différentes dans cette colonne booléenne: "f" et "m" tous deux en miniscule.
# Tout est conforme. 

array(['f', 'm'], dtype=object)

### C - Colonne "birth" :

In [7]:
customers["birth"].sort_values().unique()
# Les dates de naissance vont de 1929 (92 ans) à 2004 (17 ans). Aucune valeur aberrante, donc.
# Par ailleurs, on ne détecte aucune erreur lexicale ou d'irrégularité.
# Tout est conforme.

array([1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004],
      dtype=int64)

## II - Nettoyage du fichier "products" :

In [8]:
products.isnull().value_counts()
# Aucune valeur manquante.

id_prod  price  categ
False    False  False    3287
dtype: int64

### A - Colonne "id_product" :

In [9]:
products["id_prod"].duplicated().sort_values()
# Pas de données dupliquées

products["id_prod"].sort_values()
# La dernière entrée "T_0" est différente des autres. Cela pourrait être anodins, mais après vérification le prix qui lui est 
# associé est aberrant (prix négatif).
# Donc suppression de l'entrée.

products=products.drop(index=731)

### B - Colonne "price" :

In [10]:
products["price"].sort_values()
# Les prix des produits vont de 0,62 à 300. Aucune valeur aberrante, la seule entrée négative a été supprimée ci-dessus.

2272      0.62
2355      0.62
370       0.66
1211      0.77
1530      0.80
         ...  
2779    233.54
1436    236.99
394     247.22
724     254.44
947     300.00
Name: price, Length: 3286, dtype: float64

### C - Colonne "categ" :

In [11]:
products["categ"].unique()
# Il n'y a que 3 entrées différentes dans la colonne "categ". Les trois semblent cohérentes.

array([0, 1, 2], dtype=int64)

## III - Nettoyage du fichier "transactions" :

In [12]:
transactions.isnull().value_counts()
# Aucune valeur manquante.

id_prod  date   session_id  client_id
False    False  False       False        337016
dtype: int64

   ### A - Vérification de la colonne "client_id" :

In [13]:
transactions["id_prod"].sort_values()
# La valeur T_0 diffère des autres valeurs.

transactions[transactions["id_prod"]=="T_0"]
# les valeurs T_0, au nombre de 200 correspondent à des tests.

# Suppression des produits "T_0" :
transactions.drop(transactions.loc[transactions["id_prod"]== "T_0"].index, inplace=True)

### B - Vérification de la colonne "date" :

In [14]:
transactions["date"].sort_values()
# Il y a un problème avec les dates qui vont du 1er mars 2021 au 28 février 2022. Ce qui signifie que TOUTES sont aberrantes !

176588    2021-03-01 00:01:07.843138
95285     2021-03-01 00:02:26.047414
216737    2021-03-01 00:02:38.311413
180767    2021-03-01 00:04:54.559692
293481    2021-03-01 00:05:18.801198
                     ...            
208601    2022-02-28 23:56:57.196281
242599    2022-02-28 23:56:57.561424
6245      2022-02-28 23:57:12.741573
301528    2022-02-28 23:59:02.381670
79892     2022-02-28 23:59:58.040472
Name: date, Length: 336816, dtype: object

In [15]:
# Conversion des données
from datetime import datetime
transactions["date"]=pd.to_datetime(transactions["date"])
transactions["date"]=transactions["date"].dt.strftime('%Y-%m-%d')
transactions.sort_values(by="date")

,id_prod,date,session_id,client_id
297154,0_1210,2021-03-01,s_342,c_2452
285743,0_1815,2021-03-01,s_181,c_6882
204887,1_369,2021-03-01,s_421,c_7621
285816,2_164,2021-03-01,s_93,c_5525
133016,1_532,2021-03-01,s_267,c_1631
...,...,...,...,...
251307,0_665,2022-02-28,s_172020,c_2047
324270,0_1104,2022-02-28,s_172164,c_6524
131077,0_1036,2022-02-28,s_171943,c_3278
282774,1_396,2022-02-28,s_172279,c_1581


### C - Vérification de la colonne "session_id" :

In [16]:
transactions["session_id"].sort_values()
# présence visible de duplicats qui peuvent correspondre à des achats groupés.

testErreur = transactions[["client_id","session_id"]].groupby(by="session_id").nunique()
testErreur[testErreur["client_id"]!=1]
# Les valeurs de session_id peuvent être associées avec plusieurs produits (panier d'achat) mais à un seul client.
# Ici chaque, aucune valeur "session_id" n'est associée à plus d'un client (ou à moins). Tout est OK !

,client_id
session_id,


### D - Vérification de la colonne "client_id" :

In [17]:
transactions["client_id"].sort_values()
# Les valeurs semblent correctes. Les doublons correspondent aux divers achats.

240351      c_1
193879      c_1
83342       c_1
26295       c_1
57766       c_1
          ...  
306617    c_999
25303     c_999
318624    c_999
147230    c_999
71963     c_999
Name: client_id, Length: 336816, dtype: object

## Export des fichiers corrigés :

In [18]:
customers.to_csv("customers2.csv", index = False, encoding="ISO-8859-1")
products.to_csv("product2.csv", index = False, encoding="ISO-8859-1")
transactions.to_csv("transactions2.csv", index = False, encoding="ISO-8859-1")